In [1]:
import torch
import onnx
from torch.profiler import profile, record_function, ProfilerActivity
from CSPDarknet_YOLOv5_final import batch_CSPDarknet, CSPDarknet

In [2]:
a = torch.randn(1, 3, 64, 64)

In [3]:
b = a.unfold(2, 16, 16).unfold(3, 16, 16)
b.shape

torch.Size([1, 3, 4, 4, 16, 16])

In [4]:
b = b.contiguous().view(-1, 3, 16, 16)

In [5]:
for batch in b:
    batch = batch.unsqueeze(0)
    print(batch.shape)

torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])
torch.Size([1, 3, 16, 16])


In [6]:
csp = batch_CSPDarknet().eval()
dummy = torch.randn(1, 3, 64, 64)
out = csp(dummy)

In [7]:
out.shape

torch.Size([1, 3])

In [8]:
csp = CSPDarknet(3, 16).eval()
dummy = torch.randn(1, 3, 64, 64)
csp.train()
csp(dummy)
csp(dummy)
csp.eval()
torch.onnx.export(csp, dummy, "csp.onnx", export_params=True, do_constant_folding=True)

In [9]:
csp = batch_CSPDarknet().eval()

In [10]:
dummy = torch.randn(1, 3, 64, 64)#.to("cuda:1")

with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True) as prof:
    with record_function("model_inference"):
        csp(dummy)

print(prof.key_averages().table(sort_by="cpu_memory_usage"))

STAGE:2023-04-28 10:21:26 2718685:2718685 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-04-28 10:21:26 2718685:2718685 ActivityProfilerController.cpp:300] Completed Stage: Collection


--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::empty         1.20%       4.488ms         1.20%       4.488ms       1.657us       1.05 Mb       1.05 Mb          2708  
                aten::empty_like         0.31%       1.169ms         0.57%       2.131ms       5.775us     360.00 Kb         256 b           369  
                    aten::conv2d         0.54%       1.995ms        81.05%     301.925ms     820.448us     312.00 Kb           0 b           368  
               aten::convolution         1.40%       5.226ms        80.52%     299.930ms     815.027us     312.00 Kb  

In [11]:
print(prof.events().table())

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                     aten::zeros         0.01%      34.000us         0.02%      77.000us      77.000us           4 b           0 b             1                                                              [[], [], [], [], []]  
                     aten::empty         0.01%      42.000us         0.01%      42.0

In [12]:
from CSPDarknet_YOLOv5_final import CSPDarknet

In [14]:
csp = CSPDarknet(3, 16)
dummy = torch.randn(1, 3, 64, 64)#.to("cuda:1")

with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True) as prof:
    with record_function("model_inference"):
        csp(dummy)

        
print(prof.key_averages().table(sort_by="cpu_memory_usage"))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::empty         3.48%     397.000us         3.48%     397.000us       1.829us     795.26 Kb     795.26 Kb           217  
                aten::batch_norm         0.70%      80.000us        21.18%       2.413ms     104.913us     289.88 Kb           0 b            23  
    aten::_batch_norm_impl_index         1.48%     169.000us        20.48%       2.333ms     101.435us     289.88 Kb           0 b            23  
         aten::native_batch_norm        15.47%       1.762ms        18.75%       2.136ms      92.870us     289.88 Kb  

STAGE:2023-04-28 10:22:15 2718685:2718685 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-04-28 10:22:15 2718685:2718685 ActivityProfilerController.cpp:300] Completed Stage: Collection
